In [ ]:
%pip install pdfplumber
%pip install pandas

In [4]:
import pdfplumber
import pandas as pd
import re
import os
from datetime import datetime

In [5]:
def extract_data(pdf_path):
    data = []

    date_pattern = re.compile(r"^\d{2}/\d{2}/\d{2}")
    table_settings = {"vertical_strategy": "explicit", "explicit_vertical_lines":[20,50, 120,190, 190,270, 270,325, 325,520], "join_x_tolerance": 4}
    pdf = pdfplumber.open(pdf_path)
    for page in pdf.pages:
        page = page.within_bbox((0, 125, page.width, 730))
        table = page.extract_table(table_settings)
        if table is None:
            continue
        for row in table:
            split_row = [cell.split("\n") if cell is not None else [""] for cell in row]
            flatten_row = [x for xs in split_row for x in xs]
            if date_pattern.match(flatten_row[0]):
                data.append(flatten_row)

    df = pd.DataFrame(data, columns=["Date", "Time", "Code/Channel", "Debit", "Credit", "Balance", "Description", "Note"])
    df["Description"] = df["Description"].str.replace(r"^DESC : ?", "", regex=True)
    df["Note"] = df["Note"].str.replace(r"^NOTE : ?", "", regex=True)
    return df

In [6]:
for file in os.listdir("."):
    try:
        if file.endswith(".pdf") and datetime.strptime(file[:-4], "%Y-%m"):
            path_to_save = f"output/{file[:-4]}.csv"
            extract_data(file).to_csv(path_to_save, index=False)
    except:
        print(f"File {file} is not eligible or error")
        continue